
MMR（Maximal Marginal Relevance）和SSE（Semantic Similarity ExampleSelector，基于上文的代码）之间的主要区别在于它们选择示例的方式和策略。以下是两者之间的核心差异：

1. 选择策略：

MMR (Maximal Marginal Relevance): MMR结合了两个目标：选择与输入最相似的示例和优化示例之间的多样性。这意味着，当MMR选择示例时，它不仅仅是基于与当前输入的相似性，还会考虑已选示例之间的差异性。如果一个示例与输入非常相似，但与已经选择的其他示例也非常相似，那么它可能会受到惩罚，不会被选择。

SSE (Semantic Similarity ExampleSelector): SSE主要是基于与输入的语义相似性来选择示例。它并不优化或考虑示例之间的多样性。这意味着，它可能会选择与输入非常相似的多个示例，即使这些示例之间可能非常相似或重复。

2. 结果的多样性：

使用MMR的结果可能会更加多样化。这是因为MMR在选择过程中优化了示例之间的差异性，从而尽量避免选择重复或相似的示例。

使用SSE的结果可能与输入更为相似，但示例之间可能会有更多的重复或相似性。

3. 应用场景：

当你希望得到与输入非常相关，但又具有一定多样性的示例时，可以使用MMR。

当你只关心与输入的最大相似性，而不太关心示例之间的多样性时，可以使用SSE。

综上所述，MMR和SSE提供了两种不同的示例选择策略，分别适用于需要多样性和只关心最大相似性的场景。

In [1]:
from langchain.prompts.example_selector import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [ ]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=2,
)
mmr_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=2,
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)
print(similar_prompt.format(adjective="worried"))